In [1]:
import numpy as np
import IPython
import IPython.display as ipd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as color
import soundfile
import scipy.signal as sig
import sklearn
import sklearn.preprocessing
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from numpy import expand_dims
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math
import os
from os import listdir
from os.path import isfile, join

In [3]:
file_path = 'nn06f_20180806_000000.wav'

# you can use scipy- import scipy.io.wavfile as siow - [samp, data] = siow.read(file_path, mmap=False)  
samples,sampling_rate=soundfile.read(file_path)

#replace this with X = pd.read_csv(r'..\annotations\training.txt', sep="\t", header=0)

X = pd.read_csv('nn_ele_hb_00-24hr_TrainingSet_v2.txt', sep="\t", header=0) 


num_rows= np.shape(X[X['filename']==file_path])[0]
ref_table=X[X['filename']==file_path]

fmax = ref_table['High Freq (Hz)'].values.tolist()
t_start=ref_table[ref_table.columns[2]][0:num_rows+1]
t_end=ref_table[ref_table.columns[2]][0:num_rows+1]+ref_table[ref_table.columns[4]][0:num_rows+1]



data = []

for i,j in zip(t_start.values.tolist(),t_end.values.tolist()):
    data.append(samples[math.floor(i)*sampling_rate:math.ceil(j)*sampling_rate])

FileNotFoundError: [Errno 2] No such file or directory: 'nn_ele_hb_00-24hr_TrainingSet_v2.txt'

In [ ]:
def normalize(x, axis=0):
    import sklearn
    return sklearn.preprocessing.minmax_scale(x, axis=axis)


def spec_plot2_one(s,samp,filename,i):

    [b,a] = sig.butter(4,[10/samp, 100/samp],'bandpass','ba') 
    data1 = sig.filtfilt(b, a, s)
    data_norm=normalize(data1)
    
    fig, axs = plt.subplots(figsize=(14, 5), facecolor='w', edgecolor='k')
        
    [f, t, Sxx] = sig.spectrogram(data_norm, samp,'hann',nperseg=samp,noverlap=samp*0.9)
    cmap = plt.get_cmap('magma')
    norm = color.Normalize(vmin=np.amin(Sxx), vmax=np.amax(Sxx))
    
    axs.pcolormesh(t, f, Sxx, cmap=cmap, norm=norm)  
    axs.set_ylim(10,70)
    
    plt.tight_layout()
    plt.ioff()
    fig.savefig('trainingimages/'+ filename + str(i)+'.png', bbox_inches='tight')
    plt.close()    

In [ ]:
try:
    os.mkdir('trainingimages')
except FileExistsError:
    pass

In [ ]:
filename=ref_table['filename'].unique()[0].strip('.wav')

for i in range(len(data)):
    spec_plot2_one(data[i],sampling_rate,filename+str(fmax[i]),i)

In [ ]:
aug = ImageDataGenerator(
		rotation_range=10,
		#width_shift_range=0.2,
		#height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
        brightness_range=[0.2,1.0],
		fill_mode="nearest")

In [ ]:
try:
    os.mkdir('transformedimages')
except FileExistsError:
    pass

In [ ]:
onlyfiles = [f for f in listdir('trainingimages') if isfile(join('trainingimages', f))]

In [ ]:
i=0
for file in onlyfiles:
    
    # load the image
    img = load_img('trainingimages/'+file)

    # convert to numpy array
    data = img_to_array(img)

    # expand dimension to one sample
    samples = expand_dims(data, 0)

    # prepare iterator
    it = aug.flow(samples, batch_size=1) #set seed if same want same transformations.

    # Get 1 sample of the augmented image
    aug_images1 = [next(it)[0].astype(np.uint8) for i in range(1)]


    im = Image.fromarray(aug_images1[0])
   
    im.save('transformedimages/'+file+'augmented'+'.png')
    
    i+=1